In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1).

# 1:

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
df = pd.read_csv("../input/../input/churn-in-telecoms-dataset/bigml_59c28831336c6604c800002a.csv", sep=",")
df.T

# 1.Постановка задачи - определить что приводит к оттоку клиентов на основе тарифов мобильной связи.
# 2.Наш таргет - столбец "churn", задача - категорильная т.к таргет представляет собой категориальный тип данных.

In [ ]:
df.info()

In [ ]:
df.groupby("churn")["phone number"].count().plot(kind='bar') 

# 3.Из диаграммы видно, что в выборке большая часть клиентов - это те, которые отказались от услуг оператора, отсюда можем сделать вывод, что распределение не нормальное.

# 4:

In [ ]:
df["state"].value_counts()

In [ ]:
df["state"].value_counts().shape[0]

# Т.к штатов очень много, а преобразовать мы их можем только с помощью getdummies(), то добавлять 51 столбец в таблицу считаю нецелесообразным т.к каждый штат содержит маленькое количество людей, причем различия по штатам очень незначительные, значит какой-либо значимой информации эта фича нам не даст. Уберем ее из таблицы.

# Телефонный номер по логике никак не влияет на таргет ибо по-сути является неким вариантом id пользователя, так что его тоже уберем из таблицы.

In [ ]:
df["area code"].value_counts()

In [ ]:
df1 = df.drop(["state", "phone number"], axis=1)
df2 = df1.copy()

df2["area code"] = df['area code'].map({408:0,415:1,510:2})
df2["international plan"] = df['international plan'].map({"no":0,"yes":1})
df2["voice mail plan"] = df['voice mail plan'].map({"no":0,"yes":1})
df2["churn"] = df['churn'].map({False:0,True:1})

df2

In [ ]:
df_without_target = df2.drop(["churn"], axis=1)
df_without_target 

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [ ]:
X = scaler.fit_transform(df_without_target )
X

# 5:

In [ ]:
from sklearn.model_selection import train_test_split

y = df2["churn"]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.3, random_state=12)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)

In [ ]:
knn.fit(X_train, y_train)

In [ ]:
y_pred = knn.predict(X_valid)
y_pred

In [ ]:
knn.score(X_valid, y_valid)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_valid, y_pred)

# 6.Можем сделать предварительный вывод, что модель с довольно хорошей точностью угадывает значения таргета, но хочется отметить что т.к выборка в большинстве своем состоит из значение таргетинговой переменной "False", то естественно увидев много "фолсов", модель легко научится их угадывать, для дальнейших выводов проведем кросс-валидацию.

# 2).

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

kf = KFold(n_splits=5, shuffle=True, random_state=42)
knn = KNeighborsClassifier(n_neighbors=5)
scores = cross_val_score(knn, X, y, 
                         cv=kf, scoring='accuracy')
print(scores)
mean_score = scores.mean()
print(mean_score)

In [ ]:
from sklearn.model_selection import GridSearchCV
knn_params = {'n_neighbors': np.arange(1, 51)}
knn_grid_cv = GridSearchCV(knn, 
                        knn_params, 
                        scoring='accuracy',
                        cv=kf)
knn_grid_cv.fit(X_train, y_train)

In [ ]:
knn_grid_cv.best_estimator_

In [ ]:
knn_grid_cv.best_score_

# Лучшее значение - 0.8898438576982105, при количестве соседей - 7.

In [ ]:
knn_grid_cv.best_params_

In [ ]:
cv_results_df = pd.DataFrame(knn_grid_cv.cv_results_)
cv_results_df

In [ ]:
plt.plot(cv_results_df['param_n_neighbors'], cv_results_df['mean_test_score'])
plt.xlabel('neighbours')
plt.ylabel('test_score')

# Так как у нас бинарный таргет, и один класс гораздо превосходит по мощности второй, то лучше бы было использовать метрику f1, сделаем тесты с ней и проанализируем полученные результаты:

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

kf = KFold(n_splits=5, shuffle=True, random_state=42)
knn_f1 = KNeighborsClassifier(n_neighbors=5)
scores = cross_val_score(knn_f1, X, y, 
                         cv=kf, scoring='f1')
print(scores)
mean_score = scores.mean()
print(mean_score)

In [ ]:
from sklearn.model_selection import GridSearchCV
knn_params = {'n_neighbors': np.arange(1, 51)}
knn_grid_cv_f1 = GridSearchCV(knn, 
                        knn_params, 
                        scoring='f1',
                        cv=kf)
knn_grid_cv_f1.fit(X_train, y_train)

In [ ]:
knn_grid_cv_f1.best_estimator_

In [ ]:
knn_grid_cv_f1.best_score_

# Лучшее значение - 0.49628078753156357, при количестве соседей - 3.

In [ ]:
knn_grid_cv_f1.best_params_

In [ ]:
cv_f1_results_df = pd.DataFrame(knn_grid_cv_f1.cv_results_)
cv_f1_results_df

In [ ]:
plt.plot(cv_f1_results_df['param_n_neighbors'], cv_f1_results_df['mean_test_score'])
plt.xlabel('neighbours')
plt.ylabel('test_score')

# Можем сделать вывод что эта метрика подходит нам намного больше т.к accuracy в случае несбалансированных классов категорий не дает адекватную картину того как наш алгоритм будет предсказывать. В свою очередь f1 не зависит от соотношения классов и подходит нам больше.

# 3).

# 1:

In [ ]:
knn_weights = KNeighborsClassifier(n_neighbors=7, weights = "distance")
knn_weights.fit(X_train, y_train)

In [ ]:
p_weights = {"p": np.linspace(1,10, 200)}
knn_weights_cv = GridSearchCV(knn_weights, p_weights, scoring="accuracy", cv = kf)
knn_weights_cv.fit(X_train, y_train)

# 2:

In [ ]:
knn_weights_cv.best_estimator_

In [ ]:
knn_weights_cv.best_score_

# Лучшее значение - 0.8906985506979993, при значении p=1.7236180904522613.

In [ ]:
knn_weights_cv_results = pd.DataFrame(knn_weights_cv.cv_results_)
knn_weights_cv_results

In [ ]:
plt.plot(knn_weights_cv_results["param_p"],knn_weights_cv_results["mean_test_score"])

# 4).

In [ ]:
from sklearn.neighbors import RadiusNeighborsClassifier

radiusnbrclsf = RadiusNeighborsClassifier(radius=5)
radiusnbrclsf.fit(X_train, y_train)

y_pred = radiusnbrclsf.predict(X_valid)
radiusnbrclsf.score(X_valid, y_valid)

In [ ]:
from sklearn.neighbors import RadiusNeighborsRegressor

radiusnbrregsor = RadiusNeighborsRegressor(radius=5)
radiusnbrregsor.fit(X_train, y_train)

y_pred = radiusnbrregsor.predict(X_valid)
radiusnbrregsor.score(X_valid, y_valid)

In [ ]:
from sklearn.neighbors import NearestCentroid

ncentroid = NearestCentroid()
ncentroid.fit(X_train, y_train)

y_pred = ncentroid.predict(X_valid)
ncentroid.score(X_valid, y_valid)